In [1]:
bRunStreetLightQueries = False

#!/usr/bin/python3

# Sample API client for StreetLight Data API
# This was tested with Python 3 on Ubuntu Linux 16.04.
# Refer to the StreetLight API documentation for more information.

import datetime as dt
import json
import requests
import sys
import time
import pandas as pd

now = dt.datetime.utcnow().isoformat()

# Edit the following two constants to a valid key and user login.
STL_KEY = 'KnsRn7hdgOQjbywguup5hBvCoMcRMNWy'
INSIGHT_LOGIN_EMAIL = 'analytics@wfrc.org'

ZONE_SET_NAME = "WasatchChoiceCenters_WFSmallAnalysisDists_v2"
MIDDLE_ZONE_SET_NAME = "Wasatch Front Bike Counters 30m Buffer"
ANALYSIS_NAME = "WF_Centers_SmallAreas_wBikeMdlFltr"

# create data ranges json object

dDateRanges = []

for y in range(2019, 2022): # end value not included in range
    # Create date ranges for each month of year in range
    for m in range(1, 13):
        dtMonthBeg = dt.datetime(y, m, 1)
        dtMonthEnd = dt.date(y + int(m / 12), (m % 12) + 1, 1) - dt.timedelta(days=1)
        dDateRanges.append(
            {
                'date_range_name': dtMonthBeg.strftime("%Y_%m"),
                'data_ranges'    : [{"start_date": dtMonthBeg.strftime("%m/%d/%Y") ,"end_date": dtMonthEnd.strftime("%m/%d/%Y")}]
            }
        )
        #print(dic)    
    # Add best months "BM" ranges for each year from April 1 through October 31
    dtBeg = dt.datetime(y, 4, 1)
    dtEnd = dt.datetime(y,10,31)
    dDateRanges.append(
        {
            'date_range_name': str(y) + "_BM",
            'data_ranges'    : [{"start_date": dtBeg.strftime("%m/%d/%Y") ,"end_date": dtEnd.strftime("%m/%d/%Y")}]
        }
    )
    # Add best year ranges for each year
    dtBeg = dt.datetime(y, 1, 1)
    dtEnd = dt.datetime(y,12,31)
    dDateRanges.append(
        {
            'date_range_name': str(y) + "_YR",
            'data_ranges'    : [{"start_date": dtBeg.strftime("%m/%d/%Y") ,"end_date": dtEnd.strftime("%m/%d/%Y")}]
        }
    )
    
dfDateRanges = pd.DataFrame(dDateRanges)
#display(dfDateRanges)

# create Modes df
dModes = [
            #['TRU','Truck'],
            ['BIC','Bicycle'],
            #['PED','Pedestrian'],
            #['BUS','Bus'],
            #['RAI','Rail'],
            #['ALL','All_Vehicles']
         ]

dfModes = pd.DataFrame(dModes, columns = ['mode_name','mode_description'])
#display(dfModes)

# create cross recordset for analysis
dfAnalysisSets = pd.DataFrame.merge(dfDateRanges, dfModes, how='cross')
dfAnalysisSets['analysis_set_name'] = ANALYSIS_NAME + '_' + dfAnalysisSets['date_range_name'].astype(str)

# rail and bus analysis periods only include April, May, September, and October of 2019 and 2020
for index, row in dfAnalysisSets.iterrows():
    if (row['mode_description']=='Rail' or row['mode_description']=='Bus'):
        if row['date_range_name'] not in ('2019_04','2019_05','2019_09','2019_10','2020_04','2020_05','2020_09','2020_10'):
            dfAnalysisSets.drop(index, inplace=True)

# add analysis toggle
dfAnalysisSets['RunQuery'] = True
#dfAnalysisSets['RunQuery'] = False
#dfAnalysisSets.loc[dfAnalysisSets['date_range_name'].str[-3:]=='_YR', 'RunQuery'] = True
#dfAnalysisSets.loc[dfAnalysisSets['analysis_set_name']=='WC_Centers_ODAnalysis_2019_01_ALL_revbins2', 'RunQuery'] = True
display(dfAnalysisSets)

def print_response(response):
    print("response code: {}".format(response.status_code))
    print("response body: {}".format(response.content))

,date_range_name,data_ranges,mode_name,mode_description,analysis_set_name,RunQuery
0,2019_01,"[{'start_date': '01/01/2019', 'end_date': '01/...",BIC,Bicycle,WF_Centers_SmallAreas_wBikeMdlFltr_2019_01,True
1,2019_02,"[{'start_date': '02/01/2019', 'end_date': '02/...",BIC,Bicycle,WF_Centers_SmallAreas_wBikeMdlFltr_2019_02,True
2,2019_03,"[{'start_date': '03/01/2019', 'end_date': '03/...",BIC,Bicycle,WF_Centers_SmallAreas_wBikeMdlFltr_2019_03,True
3,2019_04,"[{'start_date': '04/01/2019', 'end_date': '04/...",BIC,Bicycle,WF_Centers_SmallAreas_wBikeMdlFltr_2019_04,True
4,2019_05,"[{'start_date': '05/01/2019', 'end_date': '05/...",BIC,Bicycle,WF_Centers_SmallAreas_wBikeMdlFltr_2019_05,True
5,2019_06,"[{'start_date': '06/01/2019', 'end_date': '06/...",BIC,Bicycle,WF_Centers_SmallAreas_wBikeMdlFltr_2019_06,True
6,2019_07,"[{'start_date': '07/01/2019', 'end_date': '07/...",BIC,Bicycle,WF_Centers_SmallAreas_wBikeMdlFltr_2019_07,True
7,2019_08,"[{'start_date': '08/01/2019', 'end_date': '08/...",BIC,Bicycle,WF_Centers_SmallAreas_wBikeMdlFltr_2019_08,True
8,2019_09,"[{'start_date': '09/01/2019', 'end_date': '09/...",BIC,Bicycle,WF_Centers_SmallAreas_wBikeMdlFltr_2019_09,True
9,2019_10,"[{'start_date': '10/01/2019', 'end_date': '10/...",BIC,Bicycle,WF_Centers_SmallAreas_wBikeMdlFltr_2019_10,True


In [2]:
#----------------------------------------------------------------------------------------
# Create a Zone Set.
#----------------------------------------------------------------------------------------

#ZONE_SET_REQUEST = {
#    "insight_login_email": INSIGHT_LOGIN_EMAIL,
#    "zone_set_name": ZONE_SET_NAME,
#    "zones": {
#        "type": "FeatureCollection",
#        "features": [
#            { "type": "Feature", "properties": { "id": 1, "name": "Mission", "is_pass": 0, "direction": None }, "geometry": { "type": "MultiPolygon", "coordinates": [ [ [ [ -122.426698258661972, 37.769562689936315 ], [ -122.423394859041892, 37.772083876030344 ], [ -122.42225575572462, 37.770418101996292 ], [ -122.411206453547067, 37.769427623969634 ], [ -122.406991771273155, 37.769067446852375 ], [ -122.404656609472752, 37.767716767038294 ], [ -122.405169205965521, 37.762628984940662 ], [ -122.406593085112107, 37.760557752167074 ], [ -122.405795712790038, 37.758441432691242 ], [ -122.403232730326167, 37.75722564731312 ], [ -122.402549268335804, 37.751821915004783 ], [ -122.403346640657901, 37.749390106706535 ], [ -122.407561322931784, 37.748399347080543 ], [ -122.424875693354338, 37.74781389197571 ], [ -122.426698258661972, 37.769562689936315 ] ] ] ] } },
#            { "type": "Feature", "properties": { "id": 2, "name": "Financial District", "is_pass": 1, "direction": None }, "geometry": { "type": "MultiPolygon", "coordinates": [ [ [ [ -122.405425504211919, 37.798033588378779 ], [ -122.398476973976571, 37.798933675051543 ], [ -122.396654408668923, 37.799698740100226 ], [ -122.397024617247041, 37.79929370664977 ], [ -122.396768319000643, 37.798911173018389 ], [ -122.395828558763895, 37.797898574431919 ], [ -122.393607307295227, 37.799113691070012 ], [ -122.392610591892605, 37.797876072083447 ], [ -122.394233814119715, 37.79690846455324 ], [ -122.393037755636598, 37.795670808599994 ], [ -122.3913575782436, 37.796278387822021 ], [ -122.390987369665496, 37.795693311702443 ], [ -122.392439726395011, 37.794500642185817 ], [ -122.389278714689581, 37.791462623416393 ], [ -122.401182344355107, 37.781965196242638 ], [ -122.405824190372982, 37.785701296639232 ], [ -122.406222876534017, 37.785723802695827 ], [ -122.407134159187834, 37.790337399578668 ], [ -122.404058580231194, 37.790764986655553 ], [ -122.405425504211919, 37.798033588378779 ] ] ] ] } }
#        ]
#    }
#}
#
#resp = requests.post(
#    'https://insight.streetlightdata.com/api/v2/zone_sets',
#    headers = {'content-type': 'application/json', 'x-stl-key': STL_KEY},
#    data = json.dumps(ZONE_SET_REQUEST))
#
#print_response(resp)
#
#if (resp.status_code == 201):
#    print("Created Zone Set successfully.")
#else:
#    print("Error creating Zone Set.")
#    sys.exit(1)
#
#


In [3]:
#----------------------------------------------------------------------------------------
# Create an O-D Analysis.
#----------------------------------------------------------------------------------------

if bRunStreetLightQueries:
    
    for index, row in dfAnalysisSets.iterrows():
        
        if row['RunQuery']==True:

            print(row['analysis_set_name'])

            CREATE_ANALYSIS_REQUEST = {
                "insight_login_email": INSIGHT_LOGIN_EMAIL,
                "analysis_name": row['analysis_set_name'],
                "analysis_type": "OD_MF_Analysis",
                "travel_mode_type": row['mode_description'],
                "description": "",
                "oz_sets": [{"name":ZONE_SET_NAME}],
                "dz_sets": [{"name":ZONE_SET_NAME}],
                "mfz_sets": [{"name":MIDDLE_ZONE_SET_NAME}],
                "date_ranges": row['data_ranges'],
                "day_types": "All Days|17, Weekday_TuWeTh|24, Weekend_SaSu|67",
                "day_parts": "All Day|0023, Early AM (12am-6am)|0005, Peak AM (6am-9am)|0608, Mid-Day (9am-3pm)|0914, Peak PM (3pm-6pm)|1517, Late PM (6pm-12am)|1823",
                "trip_attributes": True
            }

            # traveler attributes only exist for non-Truck modes
            if (row['mode_description']!='Truck'):
                CREATE_ANALYSIS_REQUEST['traveler_attributes'] = True
                #CREATE_ANALYSIS_REQUEST['truck_weight'] = 'Medium,Heavy'

            #print(CREATE_ANALYSIS_REQUEST)
            resp = requests.post(
                'https://insight.streetlightdata.com/api/v2/analyses',
                headers = {'content-type': 'application/json', 'x-stl-key': STL_KEY},
                data = json.dumps(CREATE_ANALYSIS_REQUEST))

            print_response(resp)

            if (resp.status_code == 201):
                print("Created Analysis successfully.")
                time.sleep(10) # wait ten seconds since URL has max limit of 3 calls per 15 seconds
            else:
                print("Error creating Analysis.")
                sys.exit(1)




In [4]:
#----------------------------------------------------------------------------------------
# Check the processing status of the Analysis.
#
# Note: depending on Analysis size, Analysis processing can take minutes to hours
# (for very large Analyses). A production integration should not block UI input
# from the end user while Analyses are processing, and it should resume
# gracefully when the client application is shut down and restarted while
# Analyses process.
#----------------------------------------------------------------------------------------


#pick a month that has all modes
dfOneOfEachMode = dfAnalysisSets[dfAnalysisSets['analysis_set_name'].str.contains("WC_Centers_ODAnalysis_2019_04")]


for index, row in dfOneOfEachMode.iterrows():

    print(row['analysis_set_name'])

    CHECK_STATUS_REQUEST = {
        "analyses":[{"name": row['analysis_set_name']}]
    }

    resp = requests.post(
        'https://insight.streetlightdata.com/api/v2/analyses/status',
        headers = {'content-type': 'application/json', 'x-stl-key': STL_KEY},
        data = json.dumps(CHECK_STATUS_REQUEST))

    print_response(resp)

    if (resp.status_code != 200):
        print("Error checking Analysis Status.")
        sys.exit(1)

    json_result = json.loads(resp.text)
    analysis_status = json_result["analyses"][0]["status"]

    if (analysis_status == "Available"):
        print("Analysis is Available!")
        time.sleep(10)
        break
    elif (analysis_status == "Processing"):
        print("Analysis is processing. Trying again after 1 minute...")
        time.sleep(10)
    else:
        print(analysis_status)
        time.sleep(10)
        #print("Error running Analysis.")
        #sys.exit(1)
#    
#    # only go through first time to get available downloades
#    break

In [5]:
_keepfields         = ['Origin Zone ID','Destination Zone ID','Day Type','Day Part','Average Daily O-D Traffic (StL Index)','Avg Travel Time (sec)']
_keepfields_newname = ['OID'           ,'DID'                ,'day_type','day_part','odvolume'                             ,'odtime'               ]

dMetricInputDic = [
    ['TRU','od_comm'],
    ['BUS','od_bus' ],
    ['BIC','od_bike'],
    ['PED','od_ped' ],
    ['RAI','od_rail'],
    ['ALL','od_all' ]
]

dfMetricInputDic = pd.DataFrame(dMetricInputDic, columns = ['mode_name','metric_name'])
display(dfMetricInputDic)

#----------------------------------------------------------------------------------------
# Get the O-D results.
#----------------------------------------------------------------------------------------

#"analyses":[{"metrics":["od_comm","zone_od_comm","od_trip_comm","zone_trip_comm"]

dfImport = pd.DataFrame()

for index, row in dfAnalysisSets.iterrows():

    print("Importing " + row['analysis_set_name'] + "...")

    _metric_name = dfMetricInputDic.loc[(dfMetricInputDic['mode_name']==row['mode_name']), 'metric_name'].values[0]
    
    resp = requests.get(
        'https://insight.streetlightdata.com/api/v2/analyses/download/name/{}'.format(row['analysis_set_name']) + '/' + _metric_name,
        headers = {'x-stl-key': STL_KEY})

    # resp.text contains results in CSV format.
    #print_response(resp)

    if (resp.status_code == 200):
        # Write results to a CSV file.        
        with open("temp.csv", "w") as csv_file:
            csv_file.write(resp.text)
        
        # read csv into temporary dataframe
        dfTemp = pd.read_csv("temp.csv",usecols=_keepfields)

        # rename columns to make shorter
        dfTemp.columns = _keepfields_newname

        # add columns with data from analysis set fields
        dfTemp['mode'      ] = row['mode_name'      ]
        dfTemp['date_range'] = row['date_range_name']

        # append temporary dataframe to Import dataframe
        dfImport = pd.concat([dfImport, dfTemp])

        # pause for 2 seconds due to API restrictions
        time.sleep(2)

        print("Done.")
    else:
        print("Error fetching O-D results.")
        sys.exit(1)

,mode_name,metric_name
0,TRU,od_comm
1,BUS,od_bus
2,BIC,od_bike
3,PED,od_ped
4,RAI,od_rail
5,ALL,od_all


Importing WF_Centers_SmallAreas_wBikeMdlFltr_2019_01...
Error fetching O-D results.


SystemExit: 1

c:\Users\bhereth\AppData\Local\ESRI\conda\envs\arcgispro-py3-billenv\lib\site-packages\IPython\core\interactiveshell.py:3445: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [23]:
dfImport['day_type'] = dfImport['day_type'].str[0:1]
dfImport['day_part'] = dfImport['day_part'].str[0:1]
dfImport


,OID,DID,day_type,day_part,odvolume,odtime,mode,date_range
0,101,101,0,0,15720,770.0,TRU,2019_01
1,101,101,0,1,984,812.0,TRU,2019_01
2,101,101,0,2,3093,881.0,TRU,2019_01
3,101,101,0,3,7330,743.0,TRU,2019_01
4,101,101,0,4,3056,703.0,TRU,2019_01
...,...,...,...,...,...,...,...,...
254126,91,91,2,1,240,858.0,ALL,2021_YR
254127,91,91,2,2,1946,862.0,ALL,2021_YR
254128,91,91,2,3,6621,851.0,ALL,2021_YR
254129,91,91,2,4,3892,902.0,ALL,2021_YR


In [24]:
dfImport.to_csv(os.path.join('intermediate','centers_small_districts_w_bikemidfilters.csv'),index=False)